In [1]:
# import libraries for data analysis and imaging
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# path to dataset
train_path = r"path\C-NMC\training_data\fold_0"
val_path = r"path\C-NMC\validation_data"
test_path = r"path\C-NMC\testing_data\C-NMC_test_final_phase_data"

In [3]:
# import libraries for deep learning and image manipulation
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# standard shape of image
image_shape = (450,450,3)

In [5]:
batch_size = 16 # to train in batches

In [6]:
# set up generator for image manipulation to add diversity in dataset
image_gen = ImageDataGenerator(rotation_range=20, # rotate 20 degrees
                               width_shift_range=0.10, # shift width by max of 5%
                               height_shift_range=0.10, # shift height by max of 5%
                               rescale=1/255, # rescale image by normalzing
                               shear_range=0.1, # cutting part of image (max 10%)
                               zoom_range=0.1, # zoom by 10% max
                               horizontal_flip=True, # horizontal flip
                               fill_mode='nearest' # fill missing pixels with nearest filled value
                              )

In [7]:
# generator to get train images from directory and manipulate
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

Found 3527 images belonging to 2 classes.


In [8]:
# generator to get validation images from directory and manipulate
val_image_gen = image_gen.flow_from_directory(val_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False)

Found 1867 images belonging to 2 classes.


In [9]:
x_train, y_train = next(train_image_gen) # extract one batch
x_test, y_test = next(val_image_gen)

print(f"Train batch shape: {x_train.shape}") # (batch_size, 8, 8, 3) for RGB images

Train batch shape: (16, 450, 450, 3)


In [10]:
x_train = np.mean(x_train, axis=-1).reshape(batch_size, -1) # convert to grayscale & flatten
x_test = np.mean(x_test, axis=-1).reshape(batch_size, -1)

print(f"Flattened shape: {x_train.shape}") # (batch_size, 64)

Flattened shape: (16, 202500)


In [11]:
# import library for quantum computing
import pennylane as qml

In [12]:
# number of qubits to match number of pixels in 8x8 image
num_qubits = 16  

# initialize quantum device simulator with 16 qubits
dev = qml.device("default.qubit", wires=num_qubits)

def encode_image(image):
    """Quantum circuit to encode image data"""
    for i in range(num_qubits):
        # apply a rotation around Y-axis on each qubit
        # angle of rotation is proportional to pixel intensity (scaled by pi)
        qml.RY(image[i] * np.pi, wires=i)

@qml.qnode(dev)
def quantum_circuit(image):
    # encode image into quantum state using parameterized rotations
    encode_image(image)
    
    # measure expectation value of the Pauli-Z operator for each qubit
    # gives value between -1 and 1, representing qubit's state along Z-axis
    return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

In [13]:
# apply quantum circuit to each image in training set
# transforms classical image data into quantum feature representations
x_train_q = np.array([quantum_circuit(img) for img in x_train])

# apply same quantum transformation to test set
x_test_q = np.array([quantum_circuit(img) for img in x_test])

# print shape of resulting quantum feature vectors
# each image is now represented by vector of expectation values (one per qubit)
print(f"Quantum features shape: {x_train_q.shape}")

Quantum features shape: (16, 16)


In [14]:
def variational_circuit(params, image):
    # encode classical image into quantum states using RY rotations
    encode_image(image)
    
    # apply trainable RY rotations to each qubit
    for i in range(num_qubits):
        qml.RY(params[i], wires=i)
    
    # measure expectation value of Pauli-Z operator on first qubit - serves as output of qnn
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev)
def qnn(params, image):
    # wrap variational circuit in QNode for execution
    return variational_circuit(params, image)

def quantum_model(params, images):
    # apply qnn to batch of images
    return np.array([qnn(params, img) for img in images])

In [15]:
# import library for classical optimization routine
from scipy.optimize import minimize

In [16]:
def cost(params):
    # generate predictions from quantum model
    predictions = quantum_model(params, x_train_q)
    
    # compute mean squared error between predictions and true labels
    return np.mean((predictions - y_train) ** 2)

# initialize parameters randomly from normal distribution - one parameter per qubit for variational layer
params = np.random.randn(num_qubits)

# use BFGS optimization algorithm to minimize cost function
opt_result = minimize(cost, params, method="BFGS")

# extract optimized parameters after training
trained_params = opt_result.x

In [17]:
# use trained quantum model to make predictions on test set
y_pred_q = np.round(quantum_model(trained_params, x_test_q))

# calculate accuracy by comparing predicted labels to true labels
accuracy = np.mean(y_pred_q == y_test)

In [18]:
print(f"Quantum Model Accuracy: {accuracy * 100:.2f}%")

Quantum Model Accuracy: 100.00%
